In [ ]:
!pip install geodatasets
import xarray as xr
import rioxarray
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import dask
import numpy as np
from dask.distributed import Client
from matplotlib import colors
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import geodatasets
from shapely.ops import nearest_points
import pygmt

pd.set_option('display.max_rows', 140)

In [ ]:
def add_variable_to_gdf(gdf, da, lon360=False):
    gdf_locations = gdf.get_coordinates()
    y = xr.DataArray(gdf_locations['y'].values, dims=['locations'])
    x = xr.DataArray(gdf_locations['x'].values, dims=['locations'])
    if lon360:
        data = da.sel(lon=x+360, lat=y, method='nearest').load()
    else:
        data = da.sel(x=x, y=y, method='nearest').load()
    return data.values

# Post offices in California

In [ ]:
# state boundary accessed at https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html
states = gpd.read_file('s3://carbonplan-climate-impacts/climate-risk-comparison/location-selection/cb_2018_us_state_5m.shp')

In [ ]:
# Load in wildfire risk to communities dataset from USFS
make_zarr = False
usfs_variables = ['RPS', 'CRPS', 'WHP', 'BP', 'Exposure']
if make_zarr:
    ds = xr.Dataset()
    for var in usfs_variables:
        ds[var] = rioxarray.open_rasterio(f's3://carbonplan-climate-impacts/climate-risk-comparison/location-selection/{var}_CA.tif').load()
    ds = ds.chunk({'x': 4000, 'y': 8000})
    ds.to_zarr('s3://carbonplan-climate-impacts/climate-risk-comparison/location-selection/CA.zarr', mode='w', consolidated=True)
else:
    ds = xr.open_zarr('s3://carbonplan-climate-impacts/climate-risk-comparison/location-selection/CA.zarr')

In [ ]:
xticks = [0.0001000,0.0002154,0.0004642,0.0010000
            ,0.0021544,0.0046416,0.0100000
            ,0.0215443,0.0464159,0.1300000]
cmap = ListedColormap(["#FFF0CF","#FDDCAF","#FDCA94","#FDB27B","#FC8D59",
                       "#F26D4B","#E1452F","#C91D13","#A90000","#7F0000"])
norm = colors.BoundaryNorm(xticks, cmap.N)

In [ ]:
# Read structures from https://www.sciencebase.gov/catalog/item/59faad6be4b0531197affc83 accessed 11/16/2023
structures = gpd.read_file('s3://carbonplan-climate-impacts/climate-risk-comparison/location-selection/structures/ca/Struct_Point.shp')

In [ ]:
po = structures[structures['name'].str.contains('Post Office')]
po['Exposure'] = add_variable_to_gdf(po.to_crs('epsg:5070'), ds['Exposure'])[0]
po['WHP'] = add_variable_to_gdf(po.to_crs('epsg:5070'), ds['WHP'])[0]
po['BP'] = add_variable_to_gdf(po.to_crs('epsg:5070'), ds['BP'])[0]

In [ ]:
len(po)

In [ ]:
fig, axarr = plt.subplots()
states[states['NAME']=='California'].plot(ax=axarr, color='k', zorder=-10)
po[po['Exposure']==1].sort_values('BP').plot(ax=axarr, column='BP', legend=True, 
                                             vmin=0, markersize=5, cmap=cmap, norm=norm)

In [ ]:
requested_post_offices = po[po['Exposure']==1]

In [ ]:
request_info = requested_post_offices[['address', 'city', 'state', 'zipcode']]
request_info['latitude'] = requested_post_offices.geometry.y
request_info['longitude'] = requested_post_offices.geometry.x

request_info.columns = request_info.columns.str.upper()
request_info['ADDRESS'] = request_info['ADDRESS'].str.upper()
request_info['CITY'] = request_info['CITY'].str.upper()
request_info.reset_index(drop=True).to_csv('ca-fire.csv')

# Post offices in New York

In [ ]:
# Read structures from https://www.sciencebase.gov/catalog/item/59faad6be4b0531197affc83 accessed 11/16/2023
structures = gpd.read_file('s3://carbonplan-climate-impacts/climate-risk-comparison/location-selection/structures/ny/Struct_Point.shp')

In [ ]:
ny_state = states[states['NAME']=='New York']

In [ ]:
po = structures[structures['name'].str.contains('Post Office')]

In [ ]:
# Accessed at https://www2.census.gov/geo/tiger/TIGER2023/COUNTY/
ny_counties = gpd.read_file('s3://carbonplan-climate-impacts/climate-risk-comparison/location-selection/tl_2023_us_county/tl_2023_us_county.shp')

In [ ]:
ny_counties = ny_counties[ny_counties['STATEFP']=='36']

In [ ]:
post_office_counties = gpd.sjoin(po[['address', 'city', 'state', 'zipcode', 'geometry']], 
          ny_counties, 
          how='left', 
          predicate='intersects')

In [ ]:
requested_ny = pd.DataFrame()
for county_code in ny_counties['COUNTYFP'].values:
    requested_ny = pd.concat([requested_ny, 
                                        post_office_counties[post_office_counties['COUNTYFP']==county_code].sample(n=2, random_state=1)])

In [ ]:
request_info_ny = requested_ny[['address', 'city', 'state', 'zipcode']]
request_info_ny['latitude'] = requested_ny.geometry.y
request_info_ny['longitude'] = requested_ny.geometry.x
request_info_ny.columns = request_info_ny.columns.str.upper()

In [ ]:
fig, axarr = plt.subplots()
requested_ny.plot(ax=axarr)
ny_state.plot(ax=axarr, color='k', zorder=-10)

# Schools in New York

In [ ]:
# Access locations of public schools from https://hifld-geoplatform.opendata.arcgis.com/datasets/geoplatform::public-schools/about
schools = gpd.read_file('s3://carbonplan-climate-impacts/climate-risk-comparison/location-selection/structures/ny/PublicSchools.shp')

In [ ]:
ds = pygmt.datasets.load_earth_relief(
    resolution="01s", region=[285.71, 286.61, 40.47, 40.97]
            ).to_dataset()

In [ ]:
ny_schools = schools[schools['STATE']=='NY']
nyc_schools = ny_schools[ny_schools['COUNTY'].isin(['QUEENS', 'KINGS', 'BRONX', 'RICHMOND', 'NEW YORK'])]
nyc_schools_locations = nyc_schools.to_crs('epsg:4326').get_coordinates()
y = xr.DataArray(nyc_schools_locations['y'].values, dims=['locations'])
x = xr.DataArray(nyc_schools_locations['x'].values, dims=['locations'])
elev_schools = ds.sel(lon=x+360, lat=y, method='nearest').load()
nyc_schools['elevation'] = elev_schools.elevation.values
elementary=nyc_schools[nyc_schools['LEVEL_']=='ELEMENTARY']
# qaqc for improperly coded schools, filtering out schools that end in high school or don't start in early grades
elementary = elementary[elementary['ST_GRADE'].isin(['PK', 'KG','01', '02'])]
elementary = elementary[elementary['END_GRADE'].isin(['02', '03', '04', '05', '06', '07', '08'])]
# drop addresses that appear twice because a single building houses multiple schools
elementary = elementary.drop_duplicates(subset='ADDRESS').to_crs('epsg:4326')
# select only schools with an elevation below 10 m
elementary = elementary[elementary['elevation']<10]

In [ ]:
df = gpd.read_file(geodatasets.get_path("nybb"))
df = df.to_crs('epsg:4326')

In [ ]:
ax = df.plot(figsize=(10, 10), alpha=0.5, edgecolor="k")
elementary.sort_values('ENROLLMENT', ascending=False).plot(ax=ax)
ax.set_title('Elevation (m) - All elementary schools below 10 m elevation')

In [ ]:
# Append the nyc school locations onto the new york post office locations collected above

In [ ]:
requested_schools = elementary[['ADDRESS', 'CITY', 'STATE', 'ZIP']]
requested_schools['LATITUDE'] = elementary.geometry.y
requested_schools['LONGITUDE'] = elementary.geometry.x
requested_schools = requested_schools.rename({'ZIP': 'ZIPCODE'}, axis=1)

In [ ]:
request_info_ny = pd.concat([request_info_ny, requested_schools])

In [ ]:
request_info_ny['ADDRESS'] = request_info_ny['ADDRESS'].str.upper()
request_info_ny['CITY'] = request_info_ny['CITY'].str.upper()
request_info_ny.reset_index(drop=True).to_csv('ny-flood.csv')